In [1]:
import pymongo
import requests
from bs4 import BeautifulSoup
import time
import json

In [2]:
# Set URL to scrape
url = "http://projects.fivethirtyeight.com/2016-election-forecast/?ex_cid=rrpromo"

In [3]:
# Fetch the web page
r = requests.get(url)
html = r.content

In [4]:
# Connect to MongoDB
mc = pymongo.MongoClient()
db = mc['foo']
docs = db['docs']  # MongoDB Collection

In [5]:
docs.delete_many({'ts': {'$exists': False}})

In [6]:
# Store the document in MongoDB with a timestamp
docs.insert_one({'url': url,
                 'ts': time.time(),
                 'html': html
                 })

In [7]:
# Retrieve the document from MongoDB
retrieved_document = docs.find_one({'url': url})

# Show retrieved document
print("URL: {}".format(retrieved_document['url']))
print("Timestamp: {}").format(retrieved_document['ts'])
print("\nHTML:\n{}...".format(retrieved_document['html'][:200]))

URL: http://projects.fivethirtyeight.com/2016-election-forecast/?ex_cid=rrpromo
Timestamp: 1475616631.93

HTML:
<!DOCTYPE html>
<html lang="en"><head data-prerender-ignore=""><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no"><meta http-equiv="X-UA-Compat...


In [8]:
# Use BeautifulSoup to parse the HTML
soup = BeautifulSoup(html, 'html.parser')
candidate_text = soup.select("div.candidate-text")
candidates = candidate_text[:2]

for c in candidates:
    print([p.text.encode('utf-8') for p in c.select("p")])

['Hillary Clinton', '74.9%']
['Donald Trump', '25.1%']


In [9]:
# 'secrets.json' is a text file containing one line:
# {"nyt_key": ##CENSORED##}
with open('secrets.json', 'r') as f:
    json_str = f.read()
    secrets = json.loads(json_str)
    
# Return the contents of secrets
secrets

{u'nyt_key': u'80475398467'}

In [10]:
# Don't forget to pause between HTTP requests.
time.sleep(3)
print("This took 3 seconds longer than usual.")

This took 3 seconds longer than usual.
